In [1]:
import pyfair

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import os

In [2]:
import pyfair

from pyfair.report.base_report import FairBaseReport

In [3]:
model1 = pyfair.FairModel(name="HA1", n_simulations=10_000)
model1.input_data('Loss Event Frequency', low=20, mode=100, high=900)
model1.input_data('Probable Loss Magnitude', low=3_000_000, mode=3_500_000, high=5_000_000)
model1.calculate_all()

model2 = pyfair.FairModel(name="HA2", n_simulations=10_000)
model2.input_data('Loss Event Frequency', low=.5, mode=.7, high=.9)
model2.input_data('Probable Loss Magnitude', low=2_000_000_000, mode=3_000_000_000, high=5_000_000_000)
model2.calculate_all()

model3 = pyfair.FairModel(name="HA3", n_simulations=10_000)
model3.input_data('Loss Event Frequency', low=10, mode=15, high=16)
model3.input_data('Probable Loss Magnitude', low=500_000_000, mode=500_900_000, high=600_000_000)
model3.calculate_all()

#fr = FairReport(model1)
#fr.to_html('C:/Users/theon/Desktop/report.html')
results = model1.export_results()
node_statuses = model1.get_node_statuses()

In [ ]:
import pandas as pd


class FairSimpleReport(FairBaseReport):
    
    def __init__(self, model):
        super().__init__()
        self._model = model
        self._css = self._template_paths['css'].read_text()
        self._template = self._template_paths['simple'].read_text()
        
    def _construct_output(self):
        '''Function called by FairBaseReport.to_html()'''
        
        # Alias
        t = self._template
        # Add css
        t = t.replace('{STYLE}', self._css)
        # Add Metadata
        t = t.replace('{METADATA}', self._get_metadata_table())
        # Get logo tag
        b64 = self.base64ify(self._logo_location)
        t = t.replace('{PYTHON_LOGO}', b64)

        # Overview Table
        data = self._model.export_results()
        risk = data['Risk']
        # Shoud this be done within model?
        overview_df = pd.DataFrame(
            columns=['Risk'],
            index=['Mean', 'Stdev', 'Maximum', 'Minimum'],
            data=[risk.mean(), risk.std(), risk.max(), risk.min()]
        )
        overview_df = overview_df.applymap(lambda x: self._format_strings['Risk'].format(x))
        overview_df.loc['Simulations'] = '{0:,.0f}'.format(len(risk))
        overview_df.loc['Identifier'] = self._model.get_uuid()
        overview_html = overview_df.to_html(border=0, header=True, justify='left', classes='fair_table')
        t = t.replace('{OVERVIEW_DATAFRAME}', overview_html)
        
        # Overview Hist
        hist = self._get_distribution(self._model)
        t = t.replace('{HIST}', hist)
        
        # Overview Exceedence Curves
        exceed = self._get_exceedence_curves(self._model)
        t = t.replace('{EXCEEDENCE}', exceed)
        
        # # Detail Image
        detail_img = ''
        print(self._model.__class__.__name__)
        if self._model.__class__.__name__ == 'FairModel':
            detail_img += self._get_tree(self._model)
        if self._model.__class__.__name__ == 'FairMetaModel':
            detail_img += self._get_violins(self._model)
        t = t.replace('{PARAMETER_IMG}', detail_img)
        
        # Detail table
        detail_table = ''
        params = self._model.export_params()
        try:
            del params['Creation Datetime']
        except KeyError:
            pass
        param_df = pd.DataFrame(params).T
        param_df['distribution'] = [
            self._get_distribution_icon(self._model, target)
            for target 
            in param_df.index.values
        ]
        pd.set_option('display.max_colwidth', -1)
        detail_table += param_df.to_html(
            border=0, 
            header=True, 
            justify='left', 
            classes='fair_table',
            escape=False
        )
        t = t.replace('{PARAMETER_TABLE}', detail_table)
        print(self._format_strings)
        
        
        
        return t
        
fsr = FairSimpleReport(model1)
fsr.to_html('output.html')

raise Exception()

# Components

<section header and metadata>

<section overview>

* Summary General
    * Summary Table (mean, stdev, low, high, n)
    * hist
    * Exceedence curves
* Summary Comparison
    * Summary Table (mean, stdev, low, high, n)
    * Summary Table 2
    * Combined Hist
    * Exceedence curves
    
<section parameters>

* Model Detail
    * Parameter table with distributions
* Metamodel Detail
    * Risk table with stacked area
    
<section> and source code
* JSON

# Reports
* Compare
* Single
* Metamodel Drilldown

In [ ]:

mm = pyfair.FairMetaModel(name='MetaModel1', models=[model1, model2, model3])

fig, ax = plt.subplots(figsize=(6, 6))
# For each model, calculate and plot.
columns = mm.export_results().columns
ax.violinplot(
    mm.export_results().values,
    showmeans=False,
    showmedians=True
)
ax.axes.xaxis.set_ticklabels(columns)
ax.axes.xaxis.set_ticks([item for item in range(1, len(columns) + 1)])
ax.set_title('Metamodel Components By Risk', fontsize=20)
ax.axes.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('${x:,.0f}'))

In [ ]:
from pyfair.report.distribution import FairDistributionCurve

fdc = FairDistributionCurve([model1, model2, model3])
#fig, ax = fdc.generate_image()

fig, ax = fdc.generate_image()
fdc.generate_icon('HA1', 'Risk')

In [ ]:
from pyfair.report.exceedence import FairExceedenceCurves

fpc = FairExceedenceCurves([model1, model2, model3])
fig1, ax1 = fpc.generate_image()

In [ ]:
'''
FairModel
    Just one
FairAddititveMetaModel
    Add all the risks together and display with components
FairComparativeMetaModel
    Put risks side by side
    
Mu, Sigma for Risk

Model Params

Source
'''